In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from math import floor
from math import ceil
import csv
from statistics import stdev
from statistics import mean
import numpy as np
from ast import literal_eval

In [ ]:
RESULTS_DIR = "/home/local/ASUAD/kjatkins/plotting2"

In [ ]:
GLIB_DIR = "results"
IPML_DIR = "results"

In [ ]:
def get_data(csv_file, key, is_tuple=True, filter_func = lambda x: True):
    #print(csv_file)
    x = []
    y = []
    csv_reader = csv.DictReader(open(csv_file, "r"), delimiter=";")
    for row in csv_reader:
        _x = int(row["Interaction"])
        if True:
            x.append(_x)
            if key == 'Variational Difference (Ground Truth)' and is_tuple:
                
                y.append(literal_eval(row[key])[0])
            else:
                y.append(float(row[key]))

            
    return x, y



In [ ]:
KEYS = {
    
    "LAO (Aggregate Success Rate)": "Success Rate",
    "Variational Difference (Ground Truth)": "Variational Difference",
    #"Elapsed Time": "Elapsed Time"
    
}

In [ ]:
ipml_r_filepath = "%s/%s/run%d/%s/ipml_randomized.csv" % (RESULTS_DIR, IPML_DIR,4,'Explodingblocks')
ipml_s_filepath = "%s/%s/run%d/%s/ipml_sequential.csv" % (RESULTS_DIR, IPML_DIR,4,'Explodingblocks')
glib_g1_filepath = "%s/%s/run%d/%s/glib_l2_lndr.csv" % (RESULTS_DIR, GLIB_DIR, 8, domain)
print(get_data(glib_g1_filepath, 'Elapsed Time'))

In [ ]:

domainName = {'Explodingblocks':'Exploding Blocks', 'First_responders':'First Responders', 'Probabilistic_elevators':'Probabilistic Elevators', 'Tireworld':'Tireworld'}
dataList = []
for domain in domainName:
    ipml_r_filepath = ''
    ipml_s_filepath = ''
    glib_g1_filepath = ''
    glib_l2_filepath = ''
    for key in KEYS:

        data = {}

        fileprefix = KEYS[key][0]
        x,y = 0,0
        x_to_y = {}
        x_to_y_time = {}
        for i in range(0, 10):
            ipml_r_filepath = "%s/%s/run%d/%s/ipml_randomized.csv" % (RESULTS_DIR, IPML_DIR,i,domain)
            x_temp,y_temp = get_data(ipml_r_filepath, key, False)
            x_time_temp, y_time_temp = get_data(ipml_r_filepath, 'Elapsed Time')
            assert x_time_temp == x_temp
            sentinel = -1

            while(sentinel < len(x_temp)-1):
                if x_temp[sentinel+1] == 0:
                    sentinel += 1
                else:
                    break
            sentinel_time = -1
            while(sentinel_time < len(x_time_temp)-1):
                if x_time_temp[sentinel_time+1] == 0:
                    sentinel_time += 1
                else:
                    break
            
            x_temp = x_temp[sentinel:len(x_temp)]
            y_temp = y_temp[sentinel:len(y_temp)]
            x_time_temp = x_time_temp[sentinel_time:len(y_time_temp)]
            y_time_temp = y_time_temp[sentinel_time:len(y_time_temp)]
            for x in x_temp:
                if x not in x_to_y:
                    x_to_y[x] = []
            
            for x in x_time_temp:
                if x not in x_to_y_time:
                    x_to_y_time[x] = []
                
            assert len(x_temp) == len(y_temp)
            assert len(x_time_temp) == len(y_time_temp)
            
            for j in range(0, len(x_temp)):
                x_to_y[x_temp[j]].append(y_temp[j])
                x_to_y_time[x_temp[j]].append(y_time_temp[j])
        
        for x in x_to_y:
            x_to_y[x] = mean(x_to_y[x]) 
            x_to_y_time[x] = mean(x_to_y_time[x])
            
        x=[]
        y=[]
        x_time = []
        y_time = []
        for x_val in x_to_y:
            x.append(x_val)
        x = sorted(x)
        for x_val in x_to_y_time:
            x_time.append(x_val)
        x_time = sorted(x_time)
        for x_val in x_time:
            y_time.append(x_to_y_time[x_val])
        
        for x_val in x:
            y.append(x_to_y[x_val])
        #print(domain)
        #print(key)
        #print(y)
        #print()
            
        data["IPML Randomized (Ours)"] = {
            "x": x,
            "x_time": x_time,
            "y": y,
            "y_time": y_time,
            "key": KEYS[key],
            'domain': domainName[domain]
        }

        x_to_y = {}
        x_to_y_time={}
        for i in range(0, 10):

            ipml_s_filepath = "%s/%s/run%d/%s/ipml_sequential.csv" % (RESULTS_DIR, IPML_DIR, i, domain)
            
            
            x_temp,y_temp = get_data(ipml_s_filepath, key, False)
            x_time_temp, y_time_temp = get_data(ipml_s_filepath, 'Elapsed Time')

            assert len(y_time_temp) == len(y_temp)
            sentinel = -1
            while(sentinel < len(x_temp)-1):
                if x_temp[sentinel+1] == 0:
                    sentinel += 1
                else:
                    break
            while(sentinel_time < len(x_time_temp)-1):
                if x_time_temp[sentinel_time+1] == 0:
                    sentinel_time += 1
                else:
                    break
            
            x_time_temp = x_time_temp[sentinel_time:len(y_time_temp)]
            y_time_temp = y_time_temp[sentinel_time:len(y_time_temp)]
            x_temp = x_temp[sentinel:len(x_temp)]
            y_temp = y_temp[sentinel:len(y_temp)]
            for x in x_temp:
                if x not in x_to_y:
                    x_to_y[x] = []
            for x in x_time_temp:
                if x not in x_to_y_time:
                    x_to_y_time[x] = []

            assert len(x_temp) == len(y_temp)
            assert len(x_time_temp) == len(y_time_temp)
            
            
            for j in range(0, len(x_temp)):
                x_to_y[x_temp[j]].append(y_temp[j])
                
            for j in range(0, len(x_time_temp)):
                x_to_y_time[x_time_temp[j]].append(y_time_temp[j])
                
        
        for x in x_to_y_time:
            x_to_y_time[x] = mean(x_to_y_time[x])
        
        for x in x_to_y:
            x_to_y[x] = mean(x_to_y[x])
            
        x=[]
        y=[]
        x_time = []
        y_time = []
        for x_val in x_to_y:
            x.append(x_val)
        for x_val in x_to_y_time:
            x_time.append(x_val)
        x_time = sorted(x_time)
        x = sorted(x)
        for x_val in x:
            y.append(x_to_y[x_val])
        for x_val in x_time:
            y_time.append(x_to_y_time[x_val])
            
        
        data["IPML Sequential (Ours)"] = {
            "x": x,
            "x_time":x_time,
            "y": y,
            "y_time":y_time,
            "key": KEYS[key],
            'domain': domainName[domain]
        }
        
        x_to_y = {}
        x_to_y_time = {}
        for i in range(0, 10):
            glib_g1_filepath = "%s/%s/run%d/%s/glib_g1_lndr.csv" % (RESULTS_DIR, GLIB_DIR, i, domain)

            x_temp,y_temp = get_data(glib_g1_filepath, key)
            x_time_temp, y_time_temp = get_data(glib_g1_filepath, 'Elapsed Time')
            sentinel = -1
            while(sentinel < len(x_temp)-1):
                if x_temp[sentinel+1] == 0:
                    sentinel += 1
                else:
                    break
            while(sentinel_time < len(x_time_temp)-1):
                if x_time_temp[sentinel_time+1] == 0:
                    sentinel_time += 1
                else:
                    break
            
            x_time_temp = x_time_temp[sentinel_time:len(y_time_temp)]
            y_time_temp = y_time_temp[sentinel_time:len(y_time_temp)]
            x_temp = x_temp[sentinel:len(x_temp)]
            y_temp = y_temp[sentinel:len(y_temp)]
            
            for x in x_temp:
                if x not in x_to_y:
                    x_to_y[x] = []
            for x in x_time_temp:    
                if x not in x_to_y_time:
                    x_to_y_time[x] = []

                    
            assert len(x_temp) == len(y_temp)
            assert len(x_time_temp) == len(y_time_temp)
            
            
            for j in range(0, len(x_temp)):
                x_to_y[x_temp[j]].append(y_temp[j])

            for j in range(0, len(x_time_temp)):
                x_to_y_time[x_time_temp[j]].append(y_time_temp[j])
                
        
        for x in x_to_y_time:
            x_to_y_time[x] = mean(x_to_y_time[x])
        
        for x in x_to_y:
            x_to_y[x] = mean(x_to_y[x])
            
        
            
        x=[]
        y=[]
        y_time = []
        x_time = []
        for x_val in x_to_y:
            x.append(x_val)
        x = sorted(x)
        for x_val in x_to_y_time:
            x_time.append(x_val)
        x_time = sorted(x_time)
        for x_val in x:
            y.append(x_to_y[x_val])
        
        for x_val in x_time:
            y_time.append(x_to_y_time[x_val])
            
        
        #x_time, y_time = get_data(glib_g1_filepath, 'Elapsed Time')
        data["GLIB (G1)"] = {
            "x": x,
            "x_time":x_time,
            "y": y,
            "y_time": y_time,
            "key": KEYS[key],
            'domain': domainName[domain]
        }
        
        x_to_y = {}
        x_to_y_time = {}
        
        for i in range(0,10):
            glib_l2_filepath = "%s/%s/run%d/%s/glib_l2_lndr.csv" % (RESULTS_DIR, GLIB_DIR, i, domain)
            x_temp,y_temp = get_data(glib_l2_filepath, key)
            x_time_temp, y_time_temp = get_data(glib_l2_filepath, 'Elapsed Time')
            sentinel = -1
            while(sentinel < len(x_temp)-1):
                if x_temp[sentinel+1] == 0:
                    sentinel += 1
                else:
                    break
            
            while(sentinel_time < len(x_time_temp)-1):
                if x_time_temp[sentinel_time+1] == 0:
                    sentinel_time += 1
                else:
                    break
            
            x_time_temp = x_time_temp[sentinel_time:len(y_time_temp)]
            y_time_temp = y_time_temp[sentinel_time:len(y_time_temp)]
            x_temp = x_temp[sentinel:len(x_temp)]
            y_temp = y_temp[sentinel:len(y_temp)]

            for x in x_temp:
                if x not in x_to_y:
                    x_to_y[x] = []
                    
            for x in x_time_temp:
                if x not in x_to_y_time:
                    x_to_y_time[x] = []
            assert len(x_temp) == len(y_temp)
            assert len(x_time_temp) == len(y_time_temp)
            
            
            for j in range(0, len(x_temp)):
                x_to_y[x_temp[j]].append(y_temp[j])
            
            for j in range(0, len(x_time_temp)):
                x_to_y_time[x_time_temp[j]].append(y_time_temp[j])
                
        
        for x in x_to_y_time:
            x_to_y_time[x] = mean(x_to_y_time[x])
        for x in x_to_y:
            x_to_y[x] = mean(x_to_y[x])
            
        x=[]
        y=[]
        x_time = []
        y_time = []
        for x_val in x_to_y:
            x.append(x_val)
        x = sorted(x)
        for x_val in x:
            y.append(x_to_y[x_val])
        for x_val in x_to_y_time:
            x_time.append(x_val)
        x_time = sorted(x_time)
        for x_val in x_time:
            y_time.append(x_to_y_time[x_val])
            
        data["GLIB (L2)"] = {
            "x": x,
            "x_time":x_time,
            "y": y,
            "y_time": y_time,
            "key": KEYS[key],
            'domain': domainName[domain]
        }
        print(type(data))
        dataList.append(data)
    



In [ ]:
fig2 = plt.figure(constrained_layout=True, figsize=(20, 10))
ncols = len(domainName)
nrows= len(KEYS)

spec2 = gridspec.GridSpec(ncols=ncols, nrows=nrows, figure=fig2)



counter = 0
for i in range(0, nrows):
    counter = i
    for j in range(0, ncols):
        print(counter)
        data = dataList[counter]
        x,y,x_time,y_time,key,domain = data['IPML Randomized (Ours)']['x'], data['IPML Randomized (Ours)']['y'],data['IPML Randomized (Ours)']['x_time'],data['IPML Randomized (Ours)']['y_time'],data['IPML Randomized (Ours)']['key'], data['IPML Randomized (Ours)']['domain']

        f2_ax = fig2.add_subplot(spec2[i, j])
        f2_ax2 = f2_ax.twinx()
        line = [[0,100,200,300,400],[0,100,200,300,400]]
        f2_ax2.plot(x_time,y_time,'--')
        print(set(x) - set(x_time))
        if x == []:
            x = [0]
            y = [0]
        if x[len(x)-1] != 400:
            x.append(400)
            y.append(y[len(y)-1])
        
        y = np.array(y)
        
        
        x4_time, y4_time, x4,y4,key4 = data['IPML Sequential (Ours)']['x_time'],data['IPML Sequential (Ours)']['y_time'], data['IPML Sequential (Ours)']['x'], data['IPML Sequential (Ours)']['y'],data['IPML Sequential (Ours)']['key']
        
        if x4 == []:
            x4 = [0]
            y4 = [0]
        if x4[len(x4)-1] != 400:
            x4.append(400)
            y4.append(y4[len(y4)-1])
        
        y4=np.array(y4)
        
        
        f2_ax.plot(x,y,label='IPML Randomized (Ours)')
        f2_ax.plot(x4,y4,label='IPML Sequential (Ours)')

        x2_time, y2_time, x2,y2,key2 = data['GLIB (G1)']['x_time'],data['GLIB (G1)']['y_time'],data['GLIB (G1)']['x'], data['GLIB (G1)']['y'], data['GLIB (G1)']['key']
        if x2 == []:
            x2 = [0]
            y2 = [0]
        if x2[len(x2)-1] != 400:
            x2.append(400)
            y2.append(y2[len(y2)-1])
        y2 = np.array(y2)
        f2_ax.plot(x2,y2,label='GLIB (G1)')
        x3_time,y3_time,x3,y3,key3 = data['GLIB (L2)']['x_time'],data['GLIB (L2)']['y_time'], data['GLIB (L2)']['x'], data['GLIB (L2)']['y'], data['GLIB (L2)']['key']
        
        if x3 == []:
            x3 = [0]
            y3 = [0]
        
        if x3[len(x3)-1] != 400:
            x3.append(400)
            y3.append(y3[len(y3)-1])
        y3 = np.array(y3)
        f2_ax.plot(x3,y3,label='GLIB (L2)')
        
        
        
        yStdev = stdev(y)
        y2Stdev = stdev(y2)
        y3Stdev = stdev(y3)
        y4Stdev = stdev(y4)
        if i == 0:
            f2_ax.set_title(domain, fontsize=20)
        

        if key == "Aggregate Costs":
            f2_ax.set_ylim(0,60)
        else:
            f2_ax.set_ylim(-0.05, 1.05)

        
        if j == 0:
            f2_ax.set_ylabel(key, fontsize=19)
            
        else:
            f2_ax.set_yticklabels([])
        
        if i < nrows-1:
            f2_ax.set_xticklabels([])
        
        f2_ax.tick_params(axis='both', which='major', labelsize=18)
        
        f2_ax.set_xlim(0,400)
        
        f2_ax.fill_between(x, y-yStdev, y+yStdev, alpha=0.05)
        f2_ax.fill_between(x4,y4-y4Stdev,y4+y4Stdev,alpha=0.05)
        f2_ax.fill_between(x2,y2-y2Stdev, y2+y2Stdev, alpha=0.05)
        f2_ax.fill_between(x3,y3-y3Stdev, y3+y3Stdev, alpha=0.05)
        
        counter += nrows
        

fig2.legend(loc='upper center',labels=['IPML Randomized (Ours)', 'IPML Sequential (Ours)', 'GLIB (G1)', 'GLIB (L2)'], prop={'size': 18}, bbox_to_anchor=(0.5, 1.13), ncol=4, frameon=False, fancybox=False,borderpad=0)

fig2.text(0.53, -0.03, 'Interaction #', ha='center', fontsize=18)





plt.savefig(fname='plot_false.png',tight_layout=False, bbox_inches='tight')
